In [1]:
import os
import sys
!{sys.executable} -m pip install mlxtend

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten, Conv2D, MaxPooling2D # new!
from mlxtend.data import loadlocal_mnist
from random import random
from math import floor

Using TensorFlow backend.


In [2]:
X_train, y_train = loadlocal_mnist(
        images_path=os.path.abspath('') + '/emnist-letters-train-images-idx3-ubyte', 
        labels_path=os.path.abspath('') + '/emnist-letters-train-labels-idx1-ubyte')

In [3]:
X_test, y_test = loadlocal_mnist(
        images_path=os.path.abspath('') + '/emnist-letters-test-images-idx3-ubyte', 
        labels_path=os.path.abspath('') + '/emnist-letters-test-labels-idx1-ubyte')

In [4]:
X_train = X_train.reshape(124800, 28, 28, 1).astype('float32')
X_test = X_test.reshape(20800, 28, 28, 1).astype('float32')


In [5]:
X_train /= 255
X_test /= 255

In [6]:
random_idx = floor(random() * len(X_train))
X_0 = X_train[random_idx]
for i in range(len(X_0)):
  for j in range(len(X_0)):
    if X_0[j][i][0] == 0:
      print(" ", end="")
    elif X_0[j][i][0] < 0.1:
      print("x", end="")
    else:
      print("X", end="")
  print("")
print(y_train[random_idx], chr(ord('a') + y_train[random_idx] - 1))

                            
                            
                            
                            
                            
             xxx            
         xxxxxXXxx          
       xXXXXXXXXXXx         
      xxXXXXXXXXXXx         
     xxXXXXXXXXXXXXx        
     xXXXXXXXXXXXXXx        
   xxXXXXXXXXXXXXXXx        
   xXXXXXXXXXXXXXXXx        
  xXXXXXXXXXXXXXXXx         
  xXXXXXXXXXXXXXXX  xxx     
  xXXXXXXXXXxxxxx xxXXXXx   
  xXXXXXxxxxx     xXXXXXXx  
  xXXXXXxxxxxxxxXXXXXXXXXx  
  xXXXXXXXXXXXXXXXXXXXXXXx  
  xXXXXXXXXXXXXXXXXXXXXXx   
   XXXXXXXXXXXXXXXXXXXXxx   
    xxXXXXXXXXXXXXxxxx      
     xxxxxxXXXXXxxx         
                            
                            
                            
                            
                            
5 e


In [9]:
# Subtract 1 because dataset is not 0-indexed
y_train -= 1
y_test -= 1

n_classes = 26
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)


In [13]:
print(y_train[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  0.]


In [15]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_test, y_test))

Train on 124800 samples, validate on 20800 samples
Epoch 1/10
124800/124800 [==============================] - 174s 1ms/step - loss: 0.7774 - acc: 0.7620 - val_loss: 0.3009 - val_acc: 0.9033
Epoch 2/10
124800/124800 [==============================] - 175s 1ms/step - loss: 0.4230 - acc: 0.8657 - val_loss: 0.2456 - val_acc: 0.9215
Epoch 3/10
124800/124800 [==============================] - 174s 1ms/step - loss: 0.3461 - acc: 0.8876 - val_loss: 0.2261 - val_acc: 0.9262
Epoch 4/10
124800/124800 [==============================] - 172s 1ms/step - loss: 0.3072 - acc: 0.8999 - val_loss: 0.2076 - val_acc: 0.9323
Epoch 5/10
124800/124800 [==============================] - 177s 1ms/step - loss: 0.2789 - acc: 0.9088 - val_loss: 0.2001 - val_acc: 0.9346
Epoch 6/10
124800/124800 [==============================] - 173s 1ms/step - loss: 0.2552 - acc: 0.9145 - val_loss: 0.1997 - val_acc: 0.9355
Epoch 7/10
124800/124800 [==============================] - 174s 1ms/step - loss: 0.2413 - acc: 0.9195 - val_

In [ ]:
0

In [ ]:
4

In [ ]:
print(X_train[0])